<h1 style='font-size:40px'> NPL Risk Evaluation Modeling</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            This project aims the conceiving of a Machine Learning Model focused on assisting a bank on its credit approval strategy.
        </li>
        <li> 
            During the planning meetings, the business team has made two major requests concerning the nature of the model.
            <ul style='list-style-type:decimal'> 
                <li> 
                    It must be focused on predicting potential NPL's based on a set of clients' information.
                </li>
                <li> 
                    The output must be some kind of score suggesting the likelihood of the loan to become non-performing. They are not looking for an incisive "yes or no" answer.
                </li>
            </ul>
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Data Importing</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            The Data Engineers were able to provide two .csv views from the bank's database. The first one contains general information over the clients 
            and the second lists the loans they've contracted over some period of time.
        </li>
    </ul>
</div>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285413 sha256=13d642997aafe8226136e039516e2991015a50fb6d2052ee12ee40ee3eeb42a7
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from IPython.core.display import HTML

# Creating the project's SparkSession.
spark = SparkSession.builder.appName('NPL').getOrCreate()

# This tiny config enables us to scroll along the DataFrame's columns.
display(HTML("<style>pre { white-space: pre !important; }</style>"))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/25 15:50:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h3 style='font-size:30px;font-style:italic'> Clients Database</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            This dataset is comprised of general personal and professional information about the loans' clients.
        </li>    
        <li> 
            A particularity worth noting is that date columns show the negative amount of days since the given event took place. Positive numbers 
            indicate the number of days since the occurence ceased to exist - as it might happen with unemployed borrowers in the DAYS_EMPLOYED feature.
        </li>
    </ul>
</div>

In [3]:
path_clients = '/kaggle/input/credit-card-approval-prediction/application_record.csv'

# Defining the data types from the clients dataset.
schema_clients = '''
`ID` STRING, `CODE_GENDER` STRING, `FLAG_OWN_CAR` STRING, `FLAG_OWN_REALTY` STRING, `CNT_CHILDREN` INT,
`AMT_INCOME_TOTAL` FLOAT, `NAME_INCOME_TYPE` STRING, `NAME_EDUCATION_TYPE` STRING, `NAME_FAMILY_STATUS` STRING, `NAME_HOUSING_TYPE` STRING,
`DAYS_BIRTH` INT, `DAYS_EMPLOYED` INT, `FLAG_MOBIL` STRING, `FLAG_WORK_PHONE` STRING, `FLAG_PHONE` STRING, `FLAG_EMAIL` STRING, 
`OCCUPATION_TYPE` STRING, `CNT_FAM_MEMBERS` DOUBLE
'''

# Reading the database with the created schema.
df_clients = spark.read.csv(path_clients, header=True, schema=schema_clients)
df_clients.show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|         1

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Duplicates Disclaimer</h4>
<div> 
    <ul style='font-size:20px'> 
        <li> 
             Clients may not have unique rows in the dataset because the ID column identifies a loan contracted instead of a person.
        </li>
        <li> 
            Thus, I've found convenient for the project to create an ID column that assigns a code for the clients
        </li>
    </ul>
</div>

In [4]:
# Listing the `df_clients` features with the exception of ID.
features_clients = df_clients.columns
features_clients.remove('ID')
features_clients

['CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS']

In [5]:
# Note that the database's actual amount of clients is lower than its length. 
print(f'`df_clients` length: {df_clients.count()}')
print(f'Number of clients: {df_clients.dropDuplicates(features_clients).count()}')

`df_clients` length: 438557


Number of clients: 90085


<h3 style='font-size:30px;font-style:italic'> Loans Database</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            
        </li>
    </ul>
</div>

In [6]:
# 5001711
path_loans = '/kaggle/input/credit-card-approval-prediction/credit_record.csv'
schema_loans = '`ID` STRING, `MONTHS_BALANCE` INT, `STATUS` STRING'
df_loans = spark.read.csv(path_loans, header=True, schema=schema_loans)
(df_loans
 .select(df_loans.ID)
.distinct()
).show(10)
df_loans.filter((df_loans.ID=='5112591')| (df_loans.ID=='5112593')).show(60)

+-------+
|     ID|
+-------+
|5001858|
|5002083|
|5002574|
|5003166|
|5003224|
|5004169|
|5004491|
|5004785|
|5004980|
|5005588|
+-------+
only showing top 10 rows

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5112591|             0|     C|
|5112591|            -1|     C|
|5112591|            -2|     C|
|5112591|            -3|     C|
|5112591|            -4|     C|
|5112591|            -5|     C|
|5112591|            -6|     C|
|5112591|            -7|     C|
|5112591|            -8|     C|
|5112591|            -9|     C|
|5112591|           -10|     C|
|5112591|           -11|     C|
|5112591|           -12|     C|
|5112591|           -13|     0|
|5112593|             0|     C|
|5112593|            -1|     C|
|5112593|            -2|     C|
|5112593|            -3|     C|
|5112593|            -4|     C|
|5112593|            -5|     C|
|5112593|            -6|     C|
|5112593|            -7|     C|
|5112593|            -8|     C|
|5

<p style='color:red'> Criar ID's de Clientes; Regra de NPL's para clientes</p>